In [1]:
import pandas as pd
import numpy as np
import copy
import random
from math import sqrt
import scipy.stats as sts

from tqdm import tqdm_notebook

from supplement_package import game
from supplement_package import variables_pecan

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

In [3]:
from supplement_package.game.stackelberg import StackelbergPlayer

In [4]:
import gurobipy as gp

from supplement_package.gurobi_implementation.gurobi import GurobiSolution

## Data preliminaries 

In [5]:
agent_keys = [661, 1642, 2335, 2361, 2818, 3039, 3456, 3538, 4031, 4373, 4767, 5746, 6139, 7536, 7719, 7800, 7901, 7951, 8156, 8386, 8565, 9019, 9160, 9922, 9278]

dataframe_dict = dict()
for key in agent_keys:
    dataframe_dict.update({key : pd.read_csv('/Users/ishilov/Documents/risk_paper/risk_paper/data/df_{}.csv'.format(key))})

community_size = len(dataframe_dict)

for key in agent_keys:
    cond_min = (dataframe_dict[key]['demand'].quantile(0.001) <= dataframe_dict[key]['demand'])
    cond_max = (dataframe_dict[key]['demand'] <= dataframe_dict[key]['demand'].quantile(0.999))
    dataframe_dict[key] = dataframe_dict[key][cond_min & cond_max]

In [6]:
def generate_new_params(sample_size, community_size):
    res = {}

    for sample in range(sample_size):
        A_tilde = [random.uniform(0,1) for i in range(community_size)]
        B_tilde = [random.uniform(0,1) for i in range(community_size)]

        a = [random.uniform(0,1) for i in range(community_size)]
        b = [random.uniform(0,1) for i in range(community_size)]
        d = [random.uniform(0,1) for i in range(community_size)]

        #d_target = [[random.uniform(0,8) for j in range(len(probabilities))] for i in range(community_size)]
        #g_res = [[random.uniform(0,3) for j in range(len(probabilities))] for i in range(community_size)]

        #g_res = np.array(g_res)
        #d_target = np.array(d_target)

        risk_aversion = [random.uniform(0,1) for i in range(community_size)]

        res.update({sample : {'A_tilde' : A_tilde,
                        'B_tilde' : B_tilde,
                        'a' : a,
                        'b' : b,
                        'd' : d,
                        'risk_aversion' : risk_aversion}})

    res_reformed = {(i, key) : res[i][key] for i in range(sample_size) for key in res[0].keys()}
    mindx = pd.MultiIndex.from_tuples(res_reformed.keys())
    df = pd.DataFrame(list(res_reformed.values()), index = mindx)
    df.to_csv(f'../data/param_{sample_size}.csv')


In [7]:
def text_to_adj_matrix(matrix_path):
    res = []
    with open(matrix_path) as file:
        for s in file:
            string = ''.join(s.strip().strip(',').split(', '))
            lst_temp = [int(sym) for sym in string]

            res.append(lst_temp)

    return res

In [8]:
#generate_new_params(2000)

In [9]:
def read_df_param(sample_size):
    df_param = pd.read_csv(f'../data/param_{sample_size}.csv')
    df_param.rename({'Unnamed: 0' : 'Sample', 'Unnamed: 1' : 'Parameter'}, axis=1, inplace= True)
    df_param.set_index(['Sample', 'Parameter'], inplace=True)

    return df_param

In [10]:
def distribution_build(sample_size, agent_keys):
    res = {}
    
    for key in agent_keys:
        #chunks_demand = int(demand_dict[key][0].size / sample_size)
        #chunks_generation = int(solar_dict[key][0].size / sample_size)


        #probas_demand = [np.trapz(demand_dict[key][1][i * sample_size : (i + 1) * sample_size],
        #                            demand_dict[key][0][i * sample_size : (i + 1) * sample_size])
        #                            for i in range(chunks_demand)]

        #probas_generation = [np.trapz(generation_dict[key][1][i * sample_size : (i + 1) * sample_size],
        #                            generation_dict[key][0][i * sample_size : (i + 1) * sample_size])
        #                            for i in range(chunks_generation)]

        #res_demand = plt.hist(np.random.choice(demand_dict[key][0], size = sample_size, p = probabilities), bins = int(sample_size / 2))
        #probas_update = res_demand[0] / res_demand[0].sum() if key == 661 else res[661]['probabilities']
        #res.update({key : 
        #            {'values' : res_demand[1], 
        #            'probabilities' : probas_update}})

        hist_demand = plt.hist(dataframe_dict[key]['demand'], bins = sample_size)
        probas_demand, values_demand = hist_demand[0], hist_demand[1]
        probas_demand = probas_demand / probas_demand.sum()

        if 'solar' in dataframe_dict[key].columns:
            hist_solar = plt.hist(dataframe_dict[key][dataframe_dict[key]['solar'] >= 0]['solar'], bins = sample_size)
            probas_solar , values_solar  = hist_solar[0], hist_solar[1]
            probas_solar = probas_demand / probas_demand.sum()

        res.update({key : 
                    {'probas_demand' : probas_demand,
                    'values_demand' : values_demand,
                    'probas_solar' : probas_solar,
                    'values_solar' : values_solar}})

    return res 

In [11]:
def scenario_sampling(sample_size, agent_keys, main_key = 661):
    distribution = distribution_build(sample_size, agent_keys)

    probabilities = distribution[main_key]['probas_demand']

    d_target = []
    g_res = []
    for key in agent_keys:
        d_target.append(distribution[key]['values_demand'][:-1])
        g_res.append(distribution[key]['values_solar'][:-1])

    return probabilities, d_target, g_res

In [12]:
def param_input(df, index = 0):
    A_tilde = list(df.loc[index].loc['A_tilde'])
    B_tilde = list(df.loc[index].loc['B_tilde'])
    a = list(df.loc[index].loc['a'])
    b = list(df.loc[index].loc['b'])
    d = list(df.loc[index].loc['d'])
    risk_aversion = list(df.loc[index].loc['risk_aversion'])

    for i, RA in enumerate(risk_aversion):
        if RA > 0.95:
            risk_aversion[i] = risk_aversion[i] - 0.1

    return A_tilde, B_tilde, a, b, d, risk_aversion

In [13]:
param_amount = 2000

#generate_new_params(param_amount,community_size=community_size)
df_param = read_df_param(param_amount)

In [14]:
def sample_to_csv(scenario_amount, probabilities, d_target, g_res):
    pd.DataFrame(d_target).to_csv(f'../data/df_d_target_{scenario_amount}.csv')
    pd.DataFrame(g_res).to_csv(f'../data/df_g_res_{scenario_amount}.csv')
    pd.DataFrame(probabilities).to_csv(f'../data/df_probabilities_{scenario_amount}.csv')

In [15]:
def sample_from_csv(scenario_amount):
    df_d_target = pd.read_csv(f'../data/df_d_target_{scenario_amount}.csv').drop('Unnamed: 0', axis = 1)
    df_g_res = pd.read_csv(f'../data/df_g_res_{scenario_amount}.csv').drop('Unnamed: 0', axis = 1)
    df_probabilities =pd.read_csv('../data/df_probabilities_100.csv').drop('Unnamed: 0', axis = 1)
    
    probabilities = df_probabilities.values.squeeze()


    d_target = []
    for _, row in df_d_target.iterrows():
        d_target.append(row.values)

    g_res = []
    for _, row in df_g_res.iterrows():
        g_res.append(row.values)

    return probabilities, d_target, g_res
    

In [16]:
probabilities, d_target, g_res = sample_from_csv(100)
#A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param)

In [17]:
def agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res):
    agents = []
    StackelbergPlayer.community_size = community_size
    StackelbergPlayer.probabilities = probabilities

    epsilon = 0.001
    alpha = [[proba/(1 - min(risk_aversion)) for proba in probabilities] for i in range(community_size)]
    #alpha = [[0.2 for proba in probabilities] for i in range(community_size)]
    gamma = [100000 for proba in probabilities]

    j_max = [10 for i in range(community_size)]

    for i in range(community_size):
        agent = StackelbergPlayer(i, d_target[i], g_res[i], a[i], b[i], d[i], 
                    A_tilde[i], B_tilde[i], D_min[i], D_max[i], 
                    G_min[i], G_max[i], risk_aversion[i], Kappa[i], Cost[i], connection_matrix[i],
                    probabilities = probabilities,
                    alpha = alpha[i], 
                    gamma = gamma, 
                    insurance_bound=j_max[i])
        
        agents.append(agent)

    return agents

In [18]:
def vars_to_dict(model, list_vars):
    var_names = []
    
    for var in model.getVars():
        var_names.append(var.VarName)

    dict_res = {}
    for name, var in zip(var_names, list_vars):
        dict_res.update({name : var})

    return dict_res 


In [19]:
def gurobi_experiment(df_param, index, err_track, connection_matrix, probabilities, d_target, g_res, solution_type, verbosity = 0):
    A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param, index = index)

    if solution_type == 'centralized_optimistic':
        agents = agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res)

    model_1 = gp.Model()
    setup = GurobiSolution(agents=agents,
                    model = model_1,
                    solution_type=solution_type)

    model_1.setParam('OutputFlag', verbosity)

    setup.build_model()

    try:
        model_1.optimize()

        list_vars = model_1.X
        dict_vars = vars_to_dict(model_1, list_vars)
        objective_val = model_1.getObjective().getValue()
        
        
        
    except:
        err_track.append(index)
        list_vars = ['err']
        objective_val = 'err'
        dict_vars = vars_to_dict(model_1, list_vars)

    return dict_vars, objective_val, model_1

In [20]:
D_min = [0 for i in range(community_size)]
D_max = [dataframe_dict[i].demand.max() for i in dataframe_dict.keys()]

G_min = [0 for i in range(community_size)]
G_max = [dataframe_dict[i].grid.max() for i in dataframe_dict.keys()]

Kappa = [[10 if i!=j else 0 for i in range(community_size)] for j in range(community_size)]

Cost = [[1 for i in range(community_size)] for j in range(community_size)]


In [21]:
connection_matrix_2 = text_to_adj_matrix('../matrices/matrix_2.txt')

In [22]:
def experiment(probabilities, connection_matrix, d_target, g_res,
                df_param, solution_type='centralized_optimistic', verbosity = 0):

    total_results= {}
    err_track = []
    for index in tqdm_notebook(df_param.index.levels[0]):
        vars, objective, model = gurobi_experiment(df_param, index, err_track, connection_matrix, probabilities, d_target, g_res, solution_type, verbosity)
        total_results.update({index: {'vars' : vars,
                                    'objective' : objective}})


    results_vars = pd.DataFrame(data = [list(total_results[0]['vars'].values())],
                                        columns=total_results[0]['vars'].keys())

    results_vars['objective'] = total_results[0]['objective']

    for i in tqdm_notebook(range(1, len(total_results))):
        df_temp = pd.DataFrame(data = [list(total_results[i]['vars'].values())],
                                            columns=total_results[i]['vars'].keys())
                                
        df_temp['objective'] = total_results[i]['objective']

        results_vars = pd.concat([results_vars, df_temp], ignore_index=True)

    return total_results, results_vars, model

## Experiment

In [23]:
res_opt, df_opt, model_opt = experiment(probabilities, connection_matrix_2, d_target, g_res,
            df_param, solution_type='centralized_optimistic')

/var/folders/98/gq_0lybs5k55th2hc4jstmyw00m7tv/T/ipykernel_13532/2132750934.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index in tqdm_notebook(df_param.index.levels[0]):


  0%|          | 0/2000 [00:00<?, ?it/s]

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-04-02


/var/folders/98/gq_0lybs5k55th2hc4jstmyw00m7tv/T/ipykernel_13532/2132750934.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(1, len(total_results))):


  0%|          | 0/1999 [00:00<?, ?it/s]

In [24]:
df_opt

,D_0_0,D_0_1,D_0_2,D_0_3,D_0_4,D_0_5,D_0_6,D_0_7,D_0_8,D_0_9,...,u_24_91,u_24_92,u_24_93,u_24_94,u_24_95,u_24_96,u_24_97,u_24_98,u_24_99,objective
0,2.888336e-02,8.440026e-02,1.398297e-01,1.951687e-01,2.504141e-01,3.055625e-01,3.606103e-01,0.415554,0.470394,0.529166,...,1.941769e-08,1.465730e-08,2.163078e-08,1.458044e-08,2.187829e-08,1.302198e-08,1.986621e-08,3.033073e-08,3.086437e-08,1.395002
1,4.004262e-09,7.463529e-09,8.139983e-09,1.975685e-08,4.512363e-08,1.244142e-07,8.740520e-07,0.036018,0.088652,0.141344,...,2.941165e-05,2.234818e-05,3.223080e-05,2.239762e-05,3.232023e-05,2.005725e-05,2.968245e-05,4.046110e-05,4.049895e-05,-0.646954
2,3.735263e-10,1.689799e-09,4.821260e-02,1.032066e-01,1.581608e-01,2.130748e-01,2.679478e-01,0.322779,0.377569,0.432316,...,5.194807e-08,3.555443e-08,5.940394e-08,3.587835e-08,6.003435e-08,3.127193e-08,5.352265e-08,8.375391e-08,8.431324e-08,-2.573902
3,7.580877e-09,1.493271e-08,1.613973e-08,4.093163e-08,9.542607e-08,2.737211e-07,1.171450e-06,0.020100,0.069872,0.116910,...,7.249486e-08,5.171435e-08,8.062186e-08,5.167178e-08,8.059098e-08,4.526349e-08,7.238501e-08,1.045330e-07,1.045417e-07,3.278736
4,1.014804e-02,6.541044e-02,1.205502e-01,1.755616e-01,2.304383e-01,2.851740e-01,3.443389e-01,0.406856,0.469215,0.531126,...,9.626517e-08,6.678267e-08,1.100500e-07,6.591544e-08,1.103591e-07,5.790544e-08,9.811682e-08,1.457543e-07,1.464186e-07,1.801908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2.750440e-09,1.397484e-08,7.665084e-07,5.291875e-02,1.058755e-01,1.582814e-01,2.088872e-01,0.259596,0.309830,0.359881,...,3.489625e-08,2.474662e-08,3.889585e-08,2.478637e-08,3.895433e-08,2.178734e-08,3.512584e-08,4.968284e-08,4.960223e-08,1.383177
1996,3.822750e-10,4.176839e-09,5.634164e-02,1.193837e-01,1.796928e-01,2.400637e-01,2.990931e-01,0.357684,0.416290,0.475355,...,6.522665e-08,4.573316e-08,7.313858e-08,4.575790e-08,7.324710e-08,3.982972e-08,6.548244e-08,9.685523e-08,9.694309e-08,0.950987
1997,3.128643e-09,2.161061e-08,5.211283e-08,2.355379e-02,8.452993e-02,1.390284e-01,1.888958e-01,0.236921,0.283690,0.335619,...,4.528101e-07,3.308169e-07,5.006603e-07,3.320344e-07,5.011005e-07,2.933692e-07,4.551631e-07,6.449526e-07,6.454057e-07,6.022974
1998,1.165589e-08,3.960997e-02,9.436816e-02,1.576349e-01,2.195919e-01,2.805873e-01,3.414853e-01,0.401699,0.461674,0.520955,...,8.870190e-07,7.029368e-07,9.524291e-07,7.001760e-07,9.663698e-07,5.953575e-07,9.209626e-07,1.158450e-06,1.167416e-06,1.473206


In [25]:
df_opt.to_csv(f'../data/optimistic_results_{len(df_opt)}.csv')

In [26]:
def agent_utility_from_df_row(df_row, agents):
    res = []
    for agent in agents:
        eta_idx = f'eta_{agent.id}'
        agent_obj = df_row[eta_idx]

        for proba in agent.probabilities_ind:
            u_idx = f'u_{agent.id}_{proba}'
            w_idx = f'W_{agent.id}_{proba}'
            j_idx = f'J_{agent.id}_{proba}'

            agent_obj += (agent.alpha[proba] * df_row[j_idx]
                        + agent.gamma[proba] * df_row[w_idx]
                        + agent.probabilities[proba] / (1 - agent.risk_aversion) * df_row[u_idx])

        res.append(agent_obj)

    return res

In [27]:
def IC_utility_from_df_row(df_row, agents):
    res = 0

    for agent in agents:
        for proba in agent.probabilities_ind:
            j_idx = f'J_{agent.id}_{proba}'

            res += - agent.alpha[proba] * df_row[j_idx] + agent.probabilities[proba] * df_row[j_idx]

    return res

In [28]:
agent_objectives = []
for index, row in df_opt.iterrows():
    A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param, index)
    agents = agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix_2, d_target, g_res)
    agent_objectives.append(agent_utility_from_df_row(row, agents))

In [29]:
dict_df_param = df_param.to_dict()
for i in range(community_size):
    for trial_idx, trial in enumerate(agent_objectives):
        dict_df_param[f'{i}'].update({(trial_idx, 'objective') : trial[i]})

df_param = pd.DataFrame(dict_df_param).sort_index(level=0)
df_param.to_csv(f'../data/optimistic_params_{2000}_with_obj.csv')

In [30]:
IC_objective = []
for index, row in df_opt.iterrows():
    A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param, index)
    agents = agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix_2, d_target, g_res)
    IC_objective.append(IC_utility_from_df_row(row, agents))

In [33]:
IC_objective = pd.Series(IC_objective, name='IC objective optimistic')
IC_objective.to_csv('../data/IC_objective_optimistic_2000.csv')

In [34]:
IC_objective.describe()

count    2000.000000
mean       -0.345638
std         0.236447
min        -1.174773
25%        -0.502217
50%        -0.313272
75%        -0.151614
max        -0.000366
Name: IC objective optimistic, dtype: float64